# Importação das Bibliotecas

In [31]:
import os
import wfdb
import numpy as np
import pandas as pd
import csv

Passo 1: Carregar os Sinais e Anotações

Nesta etapa é carregado o sinal de ECG e as anotações associadas ao registro 230 da base MIT-BIH Arrhythmia Database.

In [ ]:
# Carregar sinais e anotações
record = wfdb.rdrecord('230', pn_dir='mitdb')
ann = wfdb.rdann('230', 'atr', pn_dir='mitdb')

### Passo 2: Identificar os Índices com Anotações de Ritmo

Aqui, encontra-se os índices onde há anotações de ritmo (não vazias) no campo `aux_note`.

In [8]:
# Identificar os índices de ritmo
rhythm_ann_idx = np.where(np.array(ann.aux_note) != '')[0]
print(rhythm_ann_idx)

[   0    7   23   39   46   54   63  154  158  177  198  200  202  208
  218  220  226  240  249  251  254  265  267  302  326  333  340  342
  349  354  370  387  390  394  460  462  468  470  496  501  506  513
  533  539  558  561  565  568  572  574  578  582  591  596  600  617
  621  623  631  633  636  638  644  647  651  654  658  660  666  670
  678  680  684  689  695  698  703  724  733  736  741  743  747  758
  771  775  778  791  800  805  808  820  837  840  843  855  863  870
  873  875  878  892  895  898  901  904  910  912  916  939  942  944
  950  952  959  966  970 1022 1026 1032 1037 1071 1074 1156 1160 1162
 1170 1172 1175 1178 1184 1187 1190 1192 1195 1206 1214 1223 1227 1230
 1235 1238 1248 1250 1252 1267 1272 1285 1289 1302 1307 1347 1355 1377
 1390 1393 1397 1402 1420 1422 1425 1428 1432 1437 1442 1444 1451 1454
 1457 1472 1483 1537 1556 1558 1563 1568 1572 1588 1596 1627 1633 1904
 1919 1925 1936 1938 1944 1947 1962 1972 1975 1978 2114 2134 2175 2228
 2245 

### Passo 3: Definir Intervalos de (PREX e N)

Neste passo, percorre-se as anotações para identificar os intervalos entre os batimentos (PREX) e (N).


In [10]:
# Definir intervalos entre PREX e N
intervalos = []
prex_start = None

for i in rhythm_ann_idx:
    note = ann.aux_note[i]
    if '(PREX' in note:
        prex_start = ann.sample[i]
    elif '(N' in note and prex_start is not None:
        n_end = ann.sample[i]
        intervalos.append((prex_start, n_end))
        prex_start = None  # Resetar para próxima ocorrência

In [13]:
print(np.array(intervalos))

[[  1545   5566]
 [  9490  11148]
 [ 13117  15336]
 [ 41436  42307]
 [ 47475  53350]
 [ 53629  53920]
 [ 55275  57731]
 [ 58035  59431]
 [ 62669  64636]
 [ 64896  65344]
 [ 67706  67979]
 [ 76655  82312]
 [ 83779  85247]
 [ 85507  87025]
 [ 88017  91852]
 [ 96237  96834]
 [ 97566 117638]
 [117969 119651]
 [120056 128273]
 [129561 130738]
 [132731 139171]
 [140801 146956]
 [147701 148678]
 [149341 150355]
 [150681 151598]
 [152612 155333]
 [156625 157534]
 [162587 163547]
 [163872 165950]
 [166278 166898]
 [167221 168729]
 [169417 170337]
 [170986 171926]
 [172283 173907]
 [174873 177219]
 [177538 178457]
 [179855 181405]
 [182141 183442]
 [189828 192389]
 [193163 194464]
 [194833 195833]
 [199090 202854]
 [203890 204459]
 [207866 210493]
 [211860 212504]
 [215985 221191]
 [221855 222464]
 [225946 228205]
 [230191 230839]
 [231155 231725]
 [235904 236570]
 [237217 237832]
 [238524 240114]
 [240447 241374]
 [248089 248753]
 [249102 250746]
 [251084 253098]
 [254988 255955]
 [271546 27256

### Passo 4: Verificar Consistência de `aux_note`

Verifica-se se o comprimento de `aux_note` é menor que o de `sample` e, se necessário, preenche-se com strings vazias.


In [14]:
# Verificação e preenchimento de `aux_note`
if len(ann.aux_note) < len(ann.sample):
    ann.aux_note = ann.aux_note + [''] * (len(ann.sample) - len(ann.aux_note))

### Passo 5: Reclassificar Batimentos 'N' como '*'

Reclassifica-se os batimentos 'N' que ocorrem dentro dos intervalos (PREX, N) como '*'.


In [16]:
# Reclassificar batimentos 'N'
for start, end in intervalos:
    for i in range(len(ann.sample)):
        if start <= ann.sample[i] <= end and ann.symbol[i] == 'N':
            ann.symbol[i] = '*'  # Novo símbolo

In [28]:
# Agora, vamos imprimir os intervalos e as anotações modificadas
print("Intervalos modificados (amostras e anotações):")

for start, end in intervalos:
    # Encontrar as amostras dentro do intervalo e mostrar as anotações modificadas
    for i in range(len(ann.sample)):
        if start <= ann.sample[i] <= end:
            print(f"Amostra: {ann.sample[i]}, Símbolo (anotação): {ann.symbol[i]}")

Intervalos modificados (amostras e anotações):
Amostra: 1545, Símbolo (anotação): +
Amostra: 1666, Símbolo (anotação): *
Amostra: 1926, Símbolo (anotação): *
Amostra: 2199, Símbolo (anotação): *
Amostra: 2467, Símbolo (anotação): *
Amostra: 2726, Símbolo (anotação): *
Amostra: 2985, Símbolo (anotação): *
Amostra: 3243, Símbolo (anotação): *
Amostra: 3502, Símbolo (anotação): *
Amostra: 3763, Símbolo (anotação): *
Amostra: 4026, Símbolo (anotação): *
Amostra: 4300, Símbolo (anotação): *
Amostra: 4570, Símbolo (anotação): *
Amostra: 4841, Símbolo (anotação): *
Amostra: 5115, Símbolo (anotação): *
Amostra: 5387, Símbolo (anotação): *
Amostra: 5566, Símbolo (anotação): +
Amostra: 9490, Símbolo (anotação): +
Amostra: 9598, Símbolo (anotação): *
Amostra: 9872, Símbolo (anotação): *
Amostra: 10156, Símbolo (anotação): *
Amostra: 10437, Símbolo (anotação): *
Amostra: 10717, Símbolo (anotação): *
Amostra: 10990, Símbolo (anotação): *
Amostra: 11148, Símbolo (anotação): +
Amostra: 13117, Símbolo

### Passo 6: Salvar as Anotações Modificadas

Agora, salva-se as novas anotações no arquivo com o nome '230_modified.atr'.


In [30]:
# Salvar a nova anotação
wfdb.wrann(record_name='230_modified', extension='atr',  # Nome do arquivo sem caminho
           sample=ann.sample, symbol=ann.symbol, aux_note=ann.aux_note,
           write_dir='/home/joaovfg/PFC-WPW/mit-bih-arrhythmia-database-1.0.0/')

Passo 7: Exportar para CSV

Exportação das amostras, símbolos e anotações auxiliares para um arquivo CSV.

In [32]:
# Definir o caminho do diretório
csv_dir = '/home/joaovfg/PFC-WPW/csv/'

# Verificar se o diretório existe, caso contrário, criar
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

# Caminho completo para o arquivo CSV
csv_filename = os.path.join(csv_dir, '230_modified.csv')

# Salvar o arquivo CSV
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['sample', 'symbol', 'aux_note'])

    # Escrever os dados no arquivo CSV
    for i in range(len(ann.sample)):
        writer.writerow([ann.sample[i], ann.symbol[i], ann.aux_note[i]])

print(f"Arquivo CSV salvo em: {csv_filename}")


Arquivo CSV salvo em: /home/joaovfg/PFC-WPW/csv/230_modified.csv
